In [ ]:
from sentence_transformers import SentenceTransformer, util
import faiss
import numpy as np
import json

# Load the fine-tuned model
retriever_model = SentenceTransformer("finance-embed-model")

# Load your corpus
with open("data/news.json", "r", encoding="utf-8") as f:
    corpus = [json.loads(line) for line in f][0]

corpus_texts = [c["summary"] for c in corpus]  # or 'body' if you have full text
corpus_meta = [{"title": c["title"], "link": c["link"]} for c in corpus]

# Encode corpus embeddings
corpus_embeddings = retriever_model.encode(corpus_texts, convert_to_numpy=True, batch_size=32, show_progress_bar=True)

# Normalize embeddings for cosine similarity
faiss.normalize_L2(corpus_embeddings)

# Create FAISS index
dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # inner product for cosine similarity
index.add(corpus_embeddings)
print(f"FAISS index with {index.ntotal} passages ready")


[{'title': 'Micron to cease chip sales to China, bitcoin sinks to 4-month low', 'publisher': 'Yahoo Finance Video', 'link': 'https://finance.yahoo.com/video/micron-cease-chip-sales-china-144527158.html', 'published_date': '2025-10-17T14:45:27+00:00', 'summary': "Market Catalysts host Allie Canal tracks several of the day's top trending stock tickers, including Micron Technology's (MU) move to stop supplying chips to data centers in China, bitcoin (BTC-USD) plunging this week to a four-month low, and Apple (AAPL) signing a deal to be the official broadcaster of Formula One (FWONK, FWONA). To watch more expert insights and analysis on the latest market action, check out more&nbsp;Market Catalysts.", 'ticker': 'AAPL', 'company_name': 'Apple Inc.', 'source': 'yahoo_finance'}, {'title': "AI can't justify its massive buildout — but doesn't have to, yet", 'publisher': 'Yahoo Finance', 'link': 'https://finance.yahoo.com/news/ai-cant-justify-its-massive-buildout--but-doesnt-have-to-yet-10001813

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.02it/s]

FAISS index with 10 passages ready


In [ ]:
def retrieve_passages(query, top_k=2):
    q_emb = retriever_model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, top_k)
    
    results = []
    for idx in I[0]:
        results.append(corpus_meta[idx] | {"text": corpus_texts[idx]})
    return results

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

gen_model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
generator = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name)

def generate_answer(query, passages, max_length=200):
    context = "\n\n".join([p["text"] for p in passages])
    prompt = f"Answer the following question using the context below:\n\nContext:\n{context}\n\nQuestion: {query}\nAnswer:"

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = generator.generate(**inputs, max_new_tokens=max_length)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return answer.replace(prompt, "").strip()

In [10]:
def finance_agent(query):
    top_passages = retrieve_passages(query, top_k=5)
    answer = generate_answer(query, top_passages)
    return answer

In [15]:
query = "What happened with Micron's chip sales to China?"
answer = finance_agent(query)
print("Answer:\n", answer)

Answer:
 (iii)
